In [6]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np

def get_prof_data(site):

    page = urlopen(site)
    soup = BeautifulSoup(page.read(),"lxml")
    table = soup.find('table', class_='infobox vcard')
    age_final = ""
    country_born_final = ""
    state_born_final = ""
    city_born_final = ""
    nationality_final = ""
    almamater_final = ""
    education_final = ""
    occupation_final = ""
    political_party_final = ""
    
    
    if table == None:
        table = soup.find('table', class_='infobox biography vcard')
    if table == None:
        table = soup.find('table', class_='infobox vcard plainlist')
        
    if table != None:

        for th_node in table.find_all('th', {'scope' : 'row'}):
            age,country,state,city,nationality,almamater,education,occupation,political_party = ["" for _ in range(9)]
            heading_list=th_node.contents
            heading_text = str(heading_list[0])
            #print(type(heading_text))
            p = re.compile(r'<.*?>')
            clean_head = p.sub('', heading_text).strip()
            #print(clean_head)
            td_node = th_node.find_next('td')
            value_list = td_node.contents
            final_list = []
            for values in value_list:
                values_text = str(values).strip()
                clean_values = p.sub(' ', values_text)
                cleaner_values= re.sub('[^A-Za-z0-9,. ]+','', clean_values).strip()
                cleaner_values = cleaner_values.strip()
                cleanest_values= re.split(r'\s{4,}', cleaner_values.strip())
                if len(cleanest_values) >1:

                    final_list.append(cleanest_values)
                else:
                    final_list.append(cleaner_values)
            
            str_list = list(filter(None, final_list))
            lst_c =    list(filter(lambda num: num != ',', str_list))

            
            if clean_head == 'Born':
                for text in lst_c:
                    if type(text) is not  list:
                        if text.startswith('age') and len(text)>3: 
                            age_extract=([text[i:i+3] for i in range(0, len(text), 3)])
                            age= age_extract[1]
                            

                if type(lst_c[-1]) is not list:
                    address = lst_c[-1].strip().split(',')
                else:
                    address = list(filter(None, lst_c[-1]))
                if len(address) > 3:
                    address = address[-3:]
                if len(address) == 3:
                    country = address[-1].strip()
                    state = address[-2].strip()
                    city = address[-3].strip()
                elif len(address) == 2:
                    country = address[-1].strip()
                    state = address[-2].strip()
                    city = ' '
                elif len(address) == 1:
                    country = address[-1].strip()
                    state = ' '
                    city = ' '
                
            if clean_head == 'Nationality':
                nationality = lst_c[0]
              

            if re.match(r'^Alma', clean_head):
                almamater = lst_c

            if clean_head == 'Education':
                education = []
                for item in lst_c:
                    if type(item) is  list:
                        for x in item:
                            education.append(x)
                    else:
                        education.append(item)
                        

            if clean_head == 'Occupation':
                occupation = []
                for item in lst_c:
                    if type(item) is  list:
                        for x in item:
                            occupation.append(x)
                    else:
                        occupation.append(item)
          

            if re.match(r'^Political', clean_head):
                political_party = lst_c[0]
            
            if age:
                age_final = age
            if country:
                country_born_final = country
            if state:
                state_born_final = state 
            if city:
                city_born_final = city 
            if nationality:
                nationality_final = nationality
            if almamater:
                almamater_final = almamater
            if education:
                education_final = education
            if occupation:
                occupation_final = occupation
            if political_party:
                political_party_final = political_party
  

        
    
    return [age_final,country_born_final,state_born_final,city_born_final,nationality_final,almamater_final,education_final,occupation_final,political_party_final] 
            





In [7]:
get_prof_data('https://en.wikipedia.org/wiki/Andrew_Delbanco')


['', '', '', '', '', '', '', '', '']

In [12]:
y_name_list = []
y_age_list = []
y_country_born_list = []
y_state_born_list = []
y_city_born_list = []
y_nationality_list = []
y_almamater_list = []
y_education_list = []
y_occupation_list = []
y_political_party_list = []
y_wiki_link_list = []
details = {}

#guest_details_df = pd.DataFrame(columns=['age','country_born','state_born','city_born','nationality','almamater','education','occupation','political_party'])
for index, row in df.iterrows():
    print (row['wiki_link'])
    if  row['wiki_link']!= 'None':
        site = row['wiki_link'].strip()
        receive_data = get_prof_data(site)
        y_name_list.append(row['guest_name'])
        y_wiki_link_list.append(row['wiki_link']) 
        y_age_list.append(receive_data[0] if receive_data is not None else [])
        y_country_born_list.append(receive_data[1] if receive_data is not None else [])
        y_state_born_list.append(receive_data[2] if receive_data is not None else [])
        y_city_born_list.append(receive_data[3] if receive_data is not None else [])
        y_nationality_list.append(receive_data[4] if receive_data is not None else [])
        y_almamater_list.append(receive_data[5] if receive_data is not None else [])
        y_education_list.append(receive_data[6] if receive_data is not None else [])
        y_occupation_list.append(receive_data[7] if receive_data is not None else [])
        y_political_party_list.append(receive_data[8] if receive_data is not None else [])
        
details['name']= y_name_list
details['wiki_link'] = y_wiki_link_list
details['age'] = y_age_list
details['country_born'] = y_country_born_list
details['state_born'] = y_state_born_list
details['city_born'] = y_city_born_list
details['nationality'] = y_nationality_list
details['almamater'] = y_almamater_list
details['education'] = y_education_list
details['occupation'] = y_occupation_list
details['political_party'] = y_political_party_list

guest_details_df = pd.DataFrame(details)
guest_details_df.to_csv('profile_details.csv',encoding='utf-8')

        

https://en.wikipedia.org/wiki/George_Clooney
https://en.wikipedia.org/wiki/Jeb_Bush
https://en.wikipedia.org/wiki/Scarlett_Johansson
https://en.wikipedia.org/wiki/Elon_Musk
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Travis_Kalanick
https://en.wikipedia.org/wiki/Amy_Schumer
https://en.wikipedia.org/wiki/Stephen_King
https://en.wikipedia.org/wiki/Emily_Blunt
https://en.wikipedia.org/wiki/Stephen_Breyer
https://en.wikipedia.org/wiki/Jake_Gyllenhaal
https://en.wikipedia.org/wiki/Tim_Cook
https://en.wikipedia.org/wiki/Kevin_Spacey
https://en.wikipedia.org/wiki/Carol_Burnett
https://en.wikipedia.org/wiki/Trevor_Noah
https://en.wikipedia.org/wiki/Ban_Ki-moon
https://en.wikipedia.org/wiki/Lupita_Nyong%27o
https://en.wikipedia.org/wiki/Bernie_Sanders
https://en.wikipedia.org/wiki/Christopher_Wheeldon
https://en.wikipedia.org/wiki/Stephen_Curry
https://en.wikipedia.org/wiki/Ted_Cruz
https://en.wikipedia.org/wiki/Donald_Trump
https://en.wikipedia.org/wiki/Ernest_Moniz
h

https://en.wikipedia.org/wiki/Laurence_Fishburne
None
https://en.wikipedia.org/wiki/Chris_Pine
https://en.wikipedia.org/wiki/Danielle_Brooks
None
https://en.wikipedia.org/wiki/Josh_Brolin
https://en.wikipedia.org/wiki/John_Dickerson_(journalist)
https://en.wikipedia.org/wiki/Erin_Brockovich
https://en.wikipedia.org/wiki/John_Travolta
https://en.wikipedia.org/wiki/Courtney_B._Vance
https://en.wikipedia.org/wiki/Amy_Cuddy
https://en.wikipedia.org/wiki/David_Schwimmer
https://en.wikipedia.org/wiki/Joel_Osteen
https://en.wikipedia.org/wiki/Phil_McGraw
https://en.wikipedia.org/wiki/Mark_Duplass
https://en.wikipedia.org/wiki/Jay_Duplass
https://en.wikipedia.org/wiki/Michael_Eric_Dyson
https://en.wikipedia.org/wiki/Michael_Strahan
https://en.wikipedia.org/wiki/Samantha_Bee
https://en.wikipedia.org/wiki/Bobby_Cannavale
https://en.wikipedia.org/wiki/Donny_Deutsch
https://en.wikipedia.org/wiki/Tina_Fey
https://en.wikipedia.org/wiki/Margot_Robbie
https://en.wikipedia.org/wiki/Will_Ferrell
https:/

None
https://en.wikipedia.org/wiki/James_Corden
https://en.wikipedia.org/wiki/Scott_Speedman
https://en.wikipedia.org/wiki/Jude_Law
https://en.wikipedia.org/wiki/Norman_Reedus
https://en.wikipedia.org/wiki/Laura_Linney
https://en.wikipedia.org/wiki/John_Leguizamo
https://en.wikipedia.org/wiki/Patrick_Wilson_(American_actor)
https://en.wikipedia.org/wiki/Gayle_King
https://en.wikipedia.org/wiki/Gary_Johnson
https://en.wikipedia.org/wiki/William_Weld
https://en.wikipedia.org/wiki/David_Duchovny
https://en.wikipedia.org/wiki/Aaron_Tveit
https://en.wikipedia.org/wiki/Bill_O%27Reilly_(political_commentator)
https://en.wikipedia.org/wiki/Anna_Chlumsky
https://en.wikipedia.org/wiki/Daniel_Radcliffe
https://en.wikipedia.org/wiki/George_Lopez
https://en.wikipedia.org/wiki/Liam_Hemsworth
https://en.wikipedia.org/wiki/Ana_Gasteyer
https://en.wikipedia.org/wiki/Demi_Lovato
https://en.wikipedia.org/wiki/Nick_Jonas
https://en.wikipedia.org/wiki/Amy_Ryan
https://en.wikipedia.org/wiki/Aaron_Paul
https

https://en.wikipedia.org/wiki/Tom_Hanks
https://en.wikipedia.org/wiki/Adam_Conover
https://en.wikipedia.org/wiki/Will_Forte
https://en.wikipedia.org/wiki/Dermot_Mulroney
https://en.wikipedia.org/wiki/Wyatt_Cenac
https://en.wikipedia.org/wiki/Joel_McHale
https://en.wikipedia.org/wiki/Abbi_Jacobson
https://en.wikipedia.org/wiki/Jon_Glaser
https://en.wikipedia.org/wiki/Mary-Louise_Parker
https://en.wikipedia.org/wiki/Pusha_T
https://en.wikipedia.org/wiki/Drew_Carey
https://en.wikipedia.org/wiki/Claire_Foy
https://en.wikipedia.org/wiki/Matt_Smith_(actor)
https://en.wikipedia.org/wiki/Ruth_Wilson
https://en.wikipedia.org/wiki/J._B._Smoove
https://en.wikipedia.org/wiki/Mel_Gibson
https://en.wikipedia.org/wiki/Luke_Bracey
https://en.wikipedia.org/wiki/Michael_Strahan
https://en.wikipedia.org/wiki/Nick_Kroll
https://en.wikipedia.org/wiki/Aaron_Eckhart
https://en.wikipedia.org/wiki/Tig_Notaro
https://en.wikipedia.org/wiki/Ron_Suskind
https://en.wikipedia.org/wiki/Viggo_Mortensen
https://en.wiki

https://en.wikipedia.org/wiki/Jackson_Galaxy
https://en.wikipedia.org/wiki/Kristen_Stewart
https://en.wikipedia.org/wiki/Maz_Jobrani
https://en.wikipedia.org/wiki/Felicity_Huffman
https://en.wikipedia.org/wiki/Jurnee_Smollett-Bell
https://en.wikipedia.org/wiki/Ewan_McGregor
https://en.wikipedia.org/wiki/Finn_Wittrock
https://en.wikipedia.org/wiki/Neil_deGrasse_Tyson
https://en.wikipedia.org/wiki/Todd_Barry
https://en.wikipedia.org/wiki/Jessica_Lange
https://en.wikipedia.org/wiki/Bassem_Youssef
https://en.wikipedia.org/wiki/Judy_Gold
https://en.wikipedia.org/wiki/Bryan_Cranston
https://en.wikipedia.org/wiki/Audra_McDonald
https://en.wikipedia.org/wiki/Ryan_Reynolds
https://en.wikipedia.org/wiki/Josh_Lucas
https://en.wikipedia.org/wiki/Andy_Daly
https://en.wikipedia.org/wiki/Glenn_Close
https://en.wikipedia.org/wiki/Michael_McKean
https://en.wikipedia.org/wiki/H._Jon_Benjamin
https://en.wikipedia.org/wiki/Lily_Tomlin
https://en.wikipedia.org/wiki/Jane_Fonda
https://en.wikipedia.org/wiki/

https://en.wikipedia.org/wiki/Sutton_Foster
https://en.wikipedia.org/wiki/Samantha_Bee
https://en.wikipedia.org/wiki/Gillian_Jacobs
https://en.wikipedia.org/wiki/Charlie_Rose
https://en.wikipedia.org/wiki/Jessica_Williams_(actress)
https://en.wikipedia.org/wiki/Matthew_McConaughey
https://en.wikipedia.org/wiki/Vanessa_Bayer
https://en.wikipedia.org/wiki/Al_Franken
https://en.wikipedia.org/wiki/Adam_Conover
https://en.wikipedia.org/wiki/Dave_Chappelle
https://en.wikipedia.org/wiki/James_Van_Der_Beek
https://en.wikipedia.org/wiki/Joe_Walsh
https://en.wikipedia.org/wiki/Anthony_Mackie
https://en.wikipedia.org/wiki/Elizabeth_Olsen
https://en.wikipedia.org/wiki/Al_Franken
https://en.wikipedia.org/wiki/Laura_Dern
https://en.wikipedia.org/wiki/Leah_Remini
https://en.wikipedia.org/wiki/Christoph_Waltz
https://en.wikipedia.org/wiki/Chris_O%27Dowd
https://en.wikipedia.org/wiki/Sean_Evans_(interviewer)
https://en.wikipedia.org/wiki/Robert_Pattinson
https://en.wikipedia.org/wiki/David_Tennant
http

https://en.wikipedia.org/wiki/Rob_Riggle
https://en.wikipedia.org/wiki/J._K._Simmons
https://en.wikipedia.org/wiki/Pete_Holmes
https://en.wikipedia.org/wiki/James_Corden
https://en.wikipedia.org/wiki/Ann_Curry
https://en.wikipedia.org/wiki/John_Dickerson_(journalist)
https://en.wikipedia.org/wiki/Eugene_Levy
https://en.wikipedia.org/wiki/Willem_Dafoe
https://en.wikipedia.org/wiki/RuPaul
https://en.wikipedia.org/wiki/Roy_Wood_Jr.
https://en.wikipedia.org/wiki/Gwyneth_Paltrow
https://en.wikipedia.org/wiki/Ben_McKenzie
https://en.wikipedia.org/wiki/Kyle_MacLachlan
https://en.wikipedia.org/wiki/Deon_Cole
https://en.wikipedia.org/wiki/Julie_Chen
https://en.wikipedia.org/wiki/Joel_Kinnaman
https://en.wikipedia.org/wiki/Jon_Favreau_(speechwriter)
https://en.wikipedia.org/wiki/Jon_Lovett
https://en.wikipedia.org/wiki/Tommy_Vietor
https://en.wikipedia.org/wiki/Jessica_Williams_(actress)
https://en.wikipedia.org/wiki/Phoebe_Robinson
https://en.wikipedia.org/wiki/Rose_McGowan
https://en.wikipedia

https://en.wikipedia.org/wiki/Trevor_Noah
https://en.wikipedia.org/wiki/Liza_Koshy
https://en.wikipedia.org/wiki/Don_Lemon
https://en.wikipedia.org/wiki/Mike_Colter
https://en.wikipedia.org/wiki/Seth_Rogen
https://en.wikipedia.org/wiki/Jessica_Chastain
https://en.wikipedia.org/wiki/Romesh_Ranganathan
https://en.wikipedia.org/wiki/Scott_Rogowsky
https://en.wikipedia.org/wiki/Hugh_Grant
https://en.wikipedia.org/wiki/Margaret_Hoover
https://en.wikipedia.org/wiki/Paul_Rudd
https://en.wikipedia.org/wiki/Ken_Jeong
https://en.wikipedia.org/wiki/Michael_Moore
https://en.wikipedia.org/wiki/Eric_Andre
https://en.wikipedia.org/wiki/Derrick_Beckles
https://en.wikipedia.org/wiki/Alexandria_Ocasio-Cortez
https://en.wikipedia.org/wiki/Gordon_Ramsay
https://en.wikipedia.org/wiki/Andrew_Rannells
https://en.wikipedia.org/wiki/Will_Arnett
https://en.wikipedia.org/wiki/Neve_Campbell
https://en.wikipedia.org/wiki/Dwayne_Johnson
https://en.wikipedia.org/wiki/Lawrence_O%27Donnell
https://en.wikipedia.org/wik

In [13]:
print(len(guest_details_df))
print(len(df))

1440
1453
